In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20201122.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice]
0,11/17/2020 18:12:54,River's Edge (1986),Jawbreaker (1999),The Dirties (2013),All About Lily Chou-Chou (2001),Brick (2005),THE SUBURBS,VERY LARGE ANTAGONIST,AVANT GARDE ANIMATION,"""SAY THE LINE, BART!"" (MOVIES WITH A FAMOUS LI...",EPIC BATTLE,MOCKBUSTERS,[Default to Nicolas Cage]
1,11/17/2020 18:15:28,Brick (2005),River's Edge (1986),Jawbreaker (1999),The Dirties (2013),All About Lily Chou-Chou (2001),EPIC BATTLE,[Default to Nicolas Cage],VERY LARGE ANTAGONIST,THE SUBURBS,"""SAY THE LINE, BART!"" (MOVIES WITH A FAMOUS LI...",MOCKBUSTERS,AVANT GARDE ANIMATION
2,11/17/2020 18:16:19,All About Lily Chou-Chou (2001),Brick (2005),River's Edge (1986),The Dirties (2013),Jawbreaker (1999),AVANT GARDE ANIMATION,MOCKBUSTERS,VERY LARGE ANTAGONIST,EPIC BATTLE,THE SUBURBS,"""SAY THE LINE, BART!"" (MOVIES WITH A FAMOUS LI...",NaN
3,11/17/2020 18:23:49,All About Lily Chou-Chou (2001),River's Edge (1986),The Dirties (2013),Jawbreaker (1999),NaN,AVANT GARDE ANIMATION,VERY LARGE ANTAGONIST,"""SAY THE LINE, BART!"" (MOVIES WITH A FAMOUS LI...",THE SUBURBS,EPIC BATTLE,[Default to Nicolas Cage],MOCKBUSTERS
4,11/17/2020 18:25:10,Brick (2005),Jawbreaker (1999),The Dirties (2013),River's Edge (1986),All About Lily Chou-Chou (2001),"""SAY THE LINE, BART!"" (MOVIES WITH A FAMOUS LI...",THE SUBURBS,VERY LARGE ANTAGONIST,MOCKBUSTERS,[Default to Nicolas Cage],EPIC BATTLE,AVANT GARDE ANIMATION
5,11/17/2020 18:33:29,Brick (2005),The Dirties (2013),Jawbreaker (1999),River's Edge (1986),NaN,VERY LARGE ANTAGONIST,MOCKBUSTERS,"""SAY THE LINE, BART!"" (MOVIES WITH A FAMOUS LI...",THE SUBURBS,NaN,NaN,NaN
6,11/17/2020 18:35:00,Jawbreaker (1999),The Dirties (2013),Brick (2005),All About Lily Chou-Chou (2001),River's Edge (1986),VERY LARGE ANTAGONIST,"""SAY THE LINE, BART!"" (MOVIES WITH A FAMOUS LI...",THE SUBURBS,MOCKBUSTERS,AVANT GARDE ANIMATION,EPIC BATTLE,[Default to Nicolas Cage]
7,11/18/2020 8:49:55,Jawbreaker (1999),All About Lily Chou-Chou (2001),The Dirties (2013),Brick (2005),River's Edge (1986),EPIC BATTLE,AVANT GARDE ANIMATION,VERY LARGE ANTAGONIST,"""SAY THE LINE, BART!"" (MOVIES WITH A FAMOUS LI...",THE SUBURBS,MOCKBUSTERS,[Default to Nicolas Cage]


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(["River's Edge (1986)", 'Jawbreaker (1999)', 'The Dirties (2013)',
       'All About Lily Chou-Chou (2001)', 'Brick (2005)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{"River's Edge (1986)": <Candidate('River's Edge (1986)')>,
 'Jawbreaker (1999)': <Candidate('Jawbreaker (1999)')>,
 'The Dirties (2013)': <Candidate('The Dirties (2013)')>,
 'All About Lily Chou-Chou (2001)': <Candidate('All About Lily Chou-Chou (2001)')>,
 'Brick (2005)': <Candidate('Brick (2005)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(River's Edge (1986), Jawbreaker (1999), The Dirties (2013), All About Lily Chou-Chou (2001), Brick (2005))>,
 <Ballot(Brick (2005), River's Edge (1986), Jawbreaker (1999), The Dirties (2013), All About Lily Chou-Chou (2001))>,
 <Ballot(All About Lily Chou-Chou (2001), Brick (2005), River's Edge (1986), The Dirties (2013), Jawbreaker (1999))>,
 <Ballot(All About Lily Chou-Chou (2001), River's Edge (1986), The Dirties (2013), Jawbreaker (1999))>,
 <Ballot(Brick (2005), Jawbreaker (1999), The Dirties (2013), River's Edge (1986), All About Lily Chou-Chou (2001))>,
 <Ballot(Brick (2005), The Dirties (2013), Jawbreaker (1999), River's Edge (1986))>,
 <Ballot(Jawbreaker (1999), The Dirties (2013), Brick (2005), All About Lily Chou-Chou (2001), River's Edge (1986))>,
 <Ballot(Jawbreaker (1999), All About Lily Chou-Chou (2001), The Dirties (2013), Brick (2005), River's Edge (1986))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                          Votes  Status
-------------------------------  -------  --------
Brick (2005)                           3  Hopeful
Jawbreaker (1999)                      2  Hopeful
All About Lily Chou-Chou (2001)        2  Hopeful
River's Edge (1986)                    1  Rejected
The Dirties (2013)                     0  Rejected

ROUND 2
Candidate                          Votes  Status
-------------------------------  -------  --------
Jawbreaker (1999)                      3  Hopeful
Brick (2005)                           3  Hopeful
All About Lily Chou-Chou (2001)        2  Rejected
River's Edge (1986)                    0  Rejected
The Dirties (2013)                     0  Rejected

FINAL RESULT
Candidate                          Votes  Status
-------------------------------  -------  --------
Jawbreaker (1999)                      4  Elected
Brick (2005)                           4  Rejected
All About Lily Chou-Chou (2001)        0  Rejected
River's Edge

In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                                             Votes  Status
--------------------------------------------------  -------  --------
VERY LARGE ANTAGONIST                                     2  Hopeful
AVANT GARDE ANIMATION                                     2  Hopeful
EPIC BATTLE                                               2  Hopeful
"SAY THE LINE, BART!" (MOVIES WITH A FAMOUS LINE.)        1  Rejected
THE SUBURBS                                               1  Rejected
MOCKBUSTERS                                               0  Rejected
[Default to Nicolas Cage]                                 0  Rejected

FINAL RESULT
Candidate                                             Votes  Status
--------------------------------------------------  -------  --------
VERY LARGE ANTAGONIST                                     4  Elected
AVANT GARDE ANIMATION                                     2  Rejected
EPIC BATTLE                                               2  Rejected
"SAY T